In [1]:
from sympy import symbols

## Define Input Parameters

In [2]:
# define camera parameters:
f, k1, k2 = symbols('f k1 k2')

In [3]:
# select intermediate variables as landmark position in camera frame:
P_X, P_Y, P_Z = symbols('P_X P_Y P_Z')

## Point-to-Pixel Projection

In [4]:
# perspective division:
p_x = -P_X/P_Z
p_y = -P_Y/P_Z

In [5]:
# get undistortion coefficent:
r_2 = p_x**2 + p_y**2

distortion_coefficient = 1 + r_2 * (k1 + k2 * r_2)

In [6]:
# conversion to pixel coordinates:
p_x_prime = f * distortion_coefficient * p_x
p_y_prime = f * distortion_coefficient * p_y

In [7]:
# with no distortion:
# p_x = P_X/P_Z
# p_y = P_Y/P_Z

# p_x_prime = f * p_x
# p_y_prime = f * p_y

## Derivative

In [8]:
from sympy import Matrix, simplify, diff

In [9]:
# define Jacobian with respect to intermediate variables:
JP = Matrix(
    [
        [diff(p_x_prime, P_X), diff(p_x_prime, P_Y), diff(p_x_prime, P_Z)],
        [diff(p_y_prime, P_X), diff(p_y_prime, P_Y), diff(p_y_prime, P_Z)]
    ]
)

In [10]:
# jacobian with respect to camera parameters:
JI = Matrix(
    [
        [diff(p_x_prime, f), diff(p_x_prime, k1), diff(p_x_prime, k2)],
        [diff(p_y_prime, f), diff(p_y_prime, k1), diff(p_y_prime, k2)]
    ]    
)

In [11]:
# Jacobian of Lie algebra:
TP = Matrix(
    [
        [1, 0, 0,    0,  P_Z, -P_Y],
        [0, 1, 0, -P_Z,    0,  P_X],
        [0, 0, 1,  P_Y, -P_X,    0]
    ]
)

In [12]:
# overall Jacobians:

# a. Jacobian for landmark position:
J_position = simplify(JP)
# b. Jacobian for camera intrinsics:
J_intrinsic = simplify(JI)
# c. Jacobian for camera pose:
J_pose = simplify(JP * TP)

## Show Result

In [13]:
from sympy import init_printing
init_printing()

In [14]:
# display Jacobian for camera pose:
J_position

⎡   ⎛     2 ⎛   2           ⎛   2      2⎞⎞      4   ⎛   2      2⎞ ⎛   2       
⎢-f⋅⎝2⋅P_X ⋅⎝P_Z ⋅k₁ + 2⋅k₂⋅⎝P_X  + P_Y ⎠⎠ + P_Z  + ⎝P_X  + P_Y ⎠⋅⎝P_Z ⋅k₁ + k
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                 5                           
⎢                                              P_Z                            
⎢                                                                             
⎢                                      ⎛   2           ⎛   2      2⎞⎞         
⎢                         -2⋅P_X⋅P_Y⋅f⋅⎝P_Z ⋅k₁ + 2⋅k₂⋅⎝P_X  + P_Y ⎠⎠         
⎢                         ────────────────────────────────────────────        
⎢                                                5                            
⎣                                             P_Z                             

  ⎛   2      2⎞⎞⎞                                         ⎛   2           ⎛   
₂⋅⎝P_X  + P_Y ⎠⎠⎠                            -2⋅P_X

In [15]:
# display Jacobian for camera intrinsics:
J_intrinsic

⎡                                                                             
⎢     ⎛   4   ⎛   2      2⎞ ⎛   2         ⎛   2      2⎞⎞⎞          ⎛   2      
⎢-P_X⋅⎝P_Z  + ⎝P_X  + P_Y ⎠⋅⎝P_Z ⋅k₁ + k₂⋅⎝P_X  + P_Y ⎠⎠⎠   -P_X⋅f⋅⎝P_X  + P_Y
⎢─────────────────────────────────────────────────────────  ──────────────────
⎢                              5                                        3     
⎢                           P_Z                                      P_Z      
⎢                                                                             
⎢                                                                             
⎢     ⎛   4   ⎛   2      2⎞ ⎛   2         ⎛   2      2⎞⎞⎞          ⎛   2      
⎢-P_Y⋅⎝P_Z  + ⎝P_X  + P_Y ⎠⋅⎝P_Z ⋅k₁ + k₂⋅⎝P_X  + P_Y ⎠⎠⎠   -P_Y⋅f⋅⎝P_X  + P_Y
⎢─────────────────────────────────────────────────────────  ──────────────────
⎢                              5                                        3     
⎣                           P_Z                     

In [16]:
# display Jacobian for landmark position:
J_pose

⎡   ⎛     2 ⎛   2           ⎛   2      2⎞⎞      4   ⎛   2      2⎞ ⎛   2       
⎢-f⋅⎝2⋅P_X ⋅⎝P_Z ⋅k₁ + 2⋅k₂⋅⎝P_X  + P_Y ⎠⎠ + P_Z  + ⎝P_X  + P_Y ⎠⋅⎝P_Z ⋅k₁ + k
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                 5                           
⎢                                              P_Z                            
⎢                                                                             
⎢                                      ⎛   2           ⎛   2      2⎞⎞         
⎢                         -2⋅P_X⋅P_Y⋅f⋅⎝P_Z ⋅k₁ + 2⋅k₂⋅⎝P_X  + P_Y ⎠⎠         
⎢                         ────────────────────────────────────────────        
⎢                                                5                            
⎣                                             P_Z                             

  ⎛   2      2⎞⎞⎞                                         ⎛   2           ⎛   
₂⋅⎝P_X  + P_Y ⎠⎠⎠                            -2⋅P_X

## Statements for C++ Eigen

In [17]:
def format_jacobian_as_cpp_eigen(J, variable, col_offset):
    """
    Format SymPy Jacobian as CPP Eigen expression
    """    
    def format_element(J, i, j):
        """
        Format Jacobian matrix element
        """
        import re
        
        # get element representation:
        element_expression = repr(simplify(J[i, j]))
        # find all element exponentials:
        element_exponentials = {e for e in re.findall(r'P_([X|Y|Z])\*{2}(\d)', element_expression)}
        
        # format all element exponentials
        for element_exponential in element_exponentials:
            c, e = element_exponential
            element_expression = re.sub(f'P_{c}\*\*{e}', f'P_{c}_{e}', element_expression)
        
        return element_exponentials, f'{variable}({i}, {j+col_offset}) = {element_expression};'
    
    # get matrix dimension:
    M, N = J.shape
    
    # init variables and assignments:
    variables = set()
    assignments = []
    
    # generate:
    for m in range(M):
        for n in range(N):
            # get variables and assignment:
            vs, a = format_element(J, m, n)
            variables = variables.union(vs)
            assignments.append(a)
    
    # format as C++ statement:
    print("// this is the analytic Jacobian expression generated by SymPy")
    # a. declarations:
    for c, e in sorted(variables, key=lambda x: f'{x[0]}{x[1]}'):
        print(f'double P_{c}_{e} = pow({c}, {e});')
    # b. Jacobian:
    print(f'\nEigen::Matrix<double, {M}, {N}> {variable};\n')
    for assignment in assignments:
        print(assignment)
    
    return None

In [18]:
# show jacobian for intermediate variables:
format_jacobian_as_cpp_eigen(J_position, 'JP', 0)

// this is the analytic Jacobian expression generated by SymPy
double P_X_2 = pow(X, 2);
double P_Y_2 = pow(Y, 2);
double P_Z_2 = pow(Z, 2);
double P_Z_4 = pow(Z, 4);
double P_Z_5 = pow(Z, 5);
double P_Z_6 = pow(Z, 6);

Eigen::Matrix<double, 2, 3> JP;

JP(0, 0) = -f*(2*P_X_2*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)) + P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)))/P_Z_5;
JP(0, 1) = -2*P_X*P_Y*f*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2))/P_Z_5;
JP(0, 2) = P_X*f*(P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)) + 2*(P_X_2 + P_Y_2)*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)))/P_Z_6;
JP(1, 0) = -2*P_X*P_Y*f*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2))/P_Z_5;
JP(1, 1) = -f*(2*P_Y_2*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)) + P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)))/P_Z_5;
JP(1, 2) = P_Y*f*(P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)) + 2*(P_X_2 + P_Y_2)*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)))/P_Z_6;


In [19]:
# show jacobian for camera intrinsics:
format_jacobian_as_cpp_eigen(J_intrinsic, 'JI', 0)

// this is the analytic Jacobian expression generated by SymPy
double P_X_2 = pow(X, 2);
double P_Y_2 = pow(Y, 2);
double P_Z_2 = pow(Z, 2);
double P_Z_3 = pow(Z, 3);
double P_Z_4 = pow(Z, 4);
double P_Z_5 = pow(Z, 5);

Eigen::Matrix<double, 2, 3> JI;

JI(0, 0) = -P_X*(P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)))/P_Z_5;
JI(0, 1) = -P_X*f*(P_X_2 + P_Y_2)/P_Z_3;
JI(0, 2) = -P_X*f*(P_X_2 + P_Y_2)**2/P_Z_5;
JI(1, 0) = -P_Y*(P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)))/P_Z_5;
JI(1, 1) = -P_Y*f*(P_X_2 + P_Y_2)/P_Z_3;
JI(1, 2) = -P_Y*f*(P_X_2 + P_Y_2)**2/P_Z_5;


In [20]:
# show jacobian for camera pose:
format_jacobian_as_cpp_eigen(J_pose, '_jacobianOplusXi', 0)

// this is the analytic Jacobian expression generated by SymPy
double P_X_2 = pow(X, 2);
double P_Y_2 = pow(Y, 2);
double P_Z_2 = pow(Z, 2);
double P_Z_4 = pow(Z, 4);
double P_Z_5 = pow(Z, 5);
double P_Z_6 = pow(Z, 6);

Eigen::Matrix<double, 2, 6> _jacobianOplusXi;

_jacobianOplusXi(0, 0) = -f*(2*P_X_2*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)) + P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)))/P_Z_5;
_jacobianOplusXi(0, 1) = -2*P_X*P_Y*f*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2))/P_Z_5;
_jacobianOplusXi(0, 2) = P_X*f*(P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)) + 2*(P_X_2 + P_Y_2)*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)))/P_Z_6;
_jacobianOplusXi(0, 3) = P_X*P_Y*f*(P_Z_4 + 2*P_Z_2*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)) + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)) + 2*(P_X_2 + P_Y_2)*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2)))/P_Z_6;
_jacobianOplusXi(0, 4) = -f*(P_X_2*(P_Z_4 + (P_X_2 + P_Y_2)*(P_Z_2*k1 + k2*(P_X_2 + P_Y_2)) + 2*(P_X_2 + P_Y_2)*(P_Z_2*k1 + 2*k2*(P_X_2 + P_Y_2))) + P_Z_2*(2*P_X_2*(P_Z_